<a href="https://colab.research.google.com/github/TetianaHrunyk/AccomodationPricesAnalysis/blob/main/GatherDataOLX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
from datetime import datetime
import pandas as pd
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Constants

In [ ]:
CITY = "warszawa"

In [ ]:
BASE = "https://www.olx.pl"
START_PAGE = f"{BASE}/nieruchomosci/mieszkania/wynajem/{CITY}/"

In [ ]:
today = datetime.strftime(datetime.today(), "%Y-%m-%dT%H:%M")

In [ ]:
OUTPUT_PATH = os.path.join("drive/MyDrive/Projects/RentalsPL/data/", f"{CITY}_{today}.csv")

# Getting all offers' links

In [ ]:
page = requests.get(START_PAGE)
soup = BeautifulSoup(page.content, 'html.parser')
# page_num = int(soup.find("ul", {"data-testid": "pagination-list"}).find_all("li")[-1].text)
page_num=25

links = soup.find_all("a")

In [ ]:
for i in range(2, page_num+1):
  page = requests.get(f"{START_PAGE}?page={i}")
  soup = BeautifulSoup(page.content, 'html.parser')
  links += soup.find_all("a")

In [ ]:
offer_links = list({x["href"] for x in links if x.get("href", "").startswith("/d/oferta")})

In [ ]:
len(offer_links)

743

# Offer

In [ ]:
all_offers_scraped = []

for offer_link in offer_links:
  offer = requests.get(f"{BASE}/{offer_link}")
  offer_soup = BeautifulSoup(offer.content, 'html.parser')

  try:
    offer_data = dict(
      added = offer_soup.find("span", {"data-cy": "ad-posted-at"}).text,
      title = offer_soup.find("h1", {"data-cy": "ad_title"}).text,
      price = offer_soup.find("div", {"data-testid": "ad-price-container"}).find("h3").text,
      tags = [x.text for x in offer_soup.find("span", {"data-cy": "ad-posted-at"}).find_parent().find_parent().find_parent().find_all("li")],
      description = offer_soup.find("div", {"data-cy": "ad_description"}).find("div").text,
      images = [x["src"] for x in offer_soup.find_all("img") if "image;" in x["src"]],
      city=CITY,
      scraped=today
    )
    all_offers_scraped.append(offer_data)
  except Exception as e:
    print(e)

'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute

# Save to file

In [ ]:
df = pd.DataFrame(all_offers_scraped)

In [ ]:
df.to_csv(OUTPUT_PATH, index=False, mode="w+")

In [ ]:
len(all_offers_scraped)

418